# 06. 스펙트럴 클러스터링

그래프 라플라시안을 이용한 비선형 클러스터링을 구현합니다.

## 학습 목표
- 유사도 그래프와 라플라시안 이해
- 고유벡터를 통한 클러스터링
- K-Means와의 비교

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

torch.manual_seed(42)

## 1. 비선형 데이터 생성

K-Means가 실패하는 동심원 형태의 데이터를 생성합니다.

In [ ]:
# 동심원 데이터 생성
n_samples = 300

# 내부 원
theta1 = torch.linspace(0, 2 * np.pi, n_samples // 2)
r1 = 1.0 + torch.randn(n_samples // 2) * 0.1
X1 = torch.stack([r1 * torch.cos(theta1), r1 * torch.sin(theta1)], dim=1)

# 외부 원
theta2 = torch.linspace(0, 2 * np.pi, n_samples // 2)
r2 = 3.0 + torch.randn(n_samples // 2) * 0.1
X2 = torch.stack([r2 * torch.cos(theta2), r2 * torch.sin(theta2)], dim=1)

X = torch.cat([X1, X2], dim=0)
true_labels = torch.cat([torch.zeros(n_samples // 2), torch.ones(n_samples // 2)])

plt.figure(figsize=(8, 8))
plt.scatter(X[:, 0], X[:, 1], c=true_labels, cmap='viridis', alpha=0.7)
plt.xlabel('x1')
plt.ylabel('x2')
plt.title('Concentric Circles (Ground Truth)')
plt.axis('equal')
plt.show()

## 2. K-Means의 실패

In [ ]:
from mlfs.classical.clustering import KMeans

# K-Means 적용
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans_labels = kmeans.fit_predict(X)

plt.figure(figsize=(8, 8))
plt.scatter(X[:, 0], X[:, 1], c=kmeans_labels, cmap='viridis', alpha=0.7)
plt.scatter(kmeans.centers[:, 0], kmeans.centers[:, 1], 
            c='red', marker='X', s=200, label='Centers')
plt.xlabel('x1')
plt.ylabel('x2')
plt.title('K-Means Clustering (Fails on Non-convex Data)')
plt.legend()
plt.axis('equal')
plt.show()

## 3. 스펙트럴 클러스터링

1. 유사도 행렬 계산 (RBF 커널)
2. 라플라시안 행렬 계산
3. 고유벡터 추출
4. 고유벡터 공간에서 K-Means

In [ ]:
from mlfs.classical.clustering import SpectralClustering

# 스펙트럴 클러스터링 적용
spectral = SpectralClustering(n_clusters=2, gamma=1.0, random_state=42)
spectral_labels = spectral.fit_predict(X)

plt.figure(figsize=(8, 8))
plt.scatter(X[:, 0], X[:, 1], c=spectral_labels, cmap='viridis', alpha=0.7)
plt.xlabel('x1')
plt.ylabel('x2')
plt.title('Spectral Clustering (Success!)')
plt.axis('equal')
plt.show()

## 4. 임베딩 공간 시각화

In [ ]:
# 스펙트럴 임베딩 확인
embedding = spectral.embedding_

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# 원본 공간
axes[0].scatter(X[:, 0], X[:, 1], c=spectral_labels, cmap='viridis', alpha=0.7)
axes[0].set_xlabel('x1')
axes[0].set_ylabel('x2')
axes[0].set_title('Original Space')
axes[0].axis('equal')

# 임베딩 공간
axes[1].scatter(embedding[:, 0], embedding[:, 1], c=spectral_labels, cmap='viridis', alpha=0.7)
axes[1].set_xlabel('Eigenvector 1')
axes[1].set_ylabel('Eigenvector 2')
axes[1].set_title('Spectral Embedding Space')

plt.tight_layout()
plt.show()

## 5. 반달 데이터셋

In [ ]:
# 반달 형태 데이터
n = 200

# 상단 반달
theta_top = torch.linspace(0, np.pi, n)
X_top = torch.stack([torch.cos(theta_top), torch.sin(theta_top)], dim=1)
X_top += torch.randn_like(X_top) * 0.05

# 하단 반달 (이동)
theta_bot = torch.linspace(0, np.pi, n)
X_bot = torch.stack([1 - torch.cos(theta_bot), 0.5 - torch.sin(theta_bot)], dim=1)
X_bot += torch.randn_like(X_bot) * 0.05

X_moons = torch.cat([X_top, X_bot], dim=0)
y_moons = torch.cat([torch.zeros(n), torch.ones(n)])

# K-Means vs Spectral
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Ground Truth
axes[0].scatter(X_moons[:, 0], X_moons[:, 1], c=y_moons, cmap='viridis', alpha=0.7)
axes[0].set_title('Ground Truth')
axes[0].axis('equal')

# K-Means
km = KMeans(n_clusters=2, random_state=42)
km_labels = km.fit_predict(X_moons)
axes[1].scatter(X_moons[:, 0], X_moons[:, 1], c=km_labels, cmap='viridis', alpha=0.7)
axes[1].set_title('K-Means (Fails)')
axes[1].axis('equal')

# Spectral
sc = SpectralClustering(n_clusters=2, gamma=5.0, random_state=42)
sc_labels = sc.fit_predict(X_moons)
axes[2].scatter(X_moons[:, 0], X_moons[:, 1], c=sc_labels, cmap='viridis', alpha=0.7)
axes[2].set_title('Spectral Clustering (Success)')
axes[2].axis('equal')

plt.tight_layout()
plt.show()

## 요약

1. **스펙트럴 클러스터링**: 그래프 구조를 활용한 클러스터링
2. **라플라시안**: L = D - W (차수 행렬 - 유사도 행렬)
3. **고유벡터**: 가장 작은 고유값에 해당하는 고유벡터 사용
4. **장점**: 비선형/비볼록 클러스터 탐지 가능
5. **단점**: O(n³) 계산 복잡도, 스케일링 어려움

다음: 차원 축소의 비선형 방법인 **LLE**